In [123]:
import pickle
import random
models = ["pretrained_SecBert", "SecBert_E5", "pretrained_SecureBert",  "SecureBert_E5", "pretrained_Gpt2","Gpt2_E5"]
# models = ["pretrained_SecBert", "SecBert", "pretrained_SecureBert",  "SecureBert", "pretrained_Gpt2","Gpt2"]
#models = ["pretrained_SecBert","pretrained_SecureBert","pretrained_Gpt2"]
current_model = models[4]
approach = 4
margin1=1.0
margin2=1.0

dir_name = "../../ics_cwe/"+current_model+"/"

with open(dir_name+'data/anchor_pos_neg_triple_{}.pkl'.format(approach), 'rb') as f:
    anchor_pos_neg_triple=pickle.load(f)


In [124]:
len(anchor_pos_neg_triple)

6588

In [125]:
anchor_pos_neg_triple[0]

(64, 689, 413)

In [126]:
import numpy as np
# Load data (text embeddings, hop distances, node pairs)
training_data = np.load('../../ics_cwe/training_data.npy')
# #text_embeddings = np.load(dir_name+'data/text_embeddings.npy')
# sweights = [x for _,_,x in training_data]
# spairs = [[int(x),int(y)] for x,y,_ in training_data]
# import random
# def shuffle_two_arrays(array1, array2):
#     combined = list(zip(array1, array2))
#     random.shuffle(combined)
#     shuffled_array1, shuffled_array2 = zip(*combined)
#     return list(shuffled_array1), list(shuffled_array2)
# spairs, sweights = shuffle_two_arrays(spairs, sweights)
# X_training = np.array(spairs)
# Y_training = np.array(sweights).reshape(-1, 1)
# # # Prepare training data
# X1_training = X_training.T[0]
# X2_training = X_training.T[1]

In [127]:
training_data.shape

(455281, 3)

In [128]:
from collections import defaultdict
pos_neg_list = defaultdict(list)
for a,p,n in anchor_pos_neg_triple:
    pos_neg_list[a].append((a,p,n))

In [129]:
hop_dist_mat = [[-1 for _ in range(1136)] for _ in range(1136)]
for  i,j,d in training_data:
    hop_dist_mat[int(i)][int(j)]=d
    hop_dist_mat[int(j)][int(i)]=d

In [130]:
hop_dist_mat[1135][203:]

[0.3076923076923077,
 0.38461538461538464,
 0.3076923076923077,
 0.38461538461538464,
 0.38461538461538464,
 0.46153846153846156,
 0.46153846153846156,
 0.38461538461538464,
 0.3076923076923077,
 0.3076923076923077,
 0.23076923076923078,
 0.6153846153846154,
 0.6923076923076923,
 0.46153846153846156,
 0.5384615384615384,
 0.5384615384615384,
 0.5384615384615384,
 0.3076923076923077,
 0.38461538461538464,
 0.46153846153846156,
 0.5384615384615384,
 0.5384615384615384,
 0.38461538461538464,
 0.38461538461538464,
 0.46153846153846156,
 0.38461538461538464,
 0.38461538461538464,
 0.46153846153846156,
 0.38461538461538464,
 0.46153846153846156,
 0.46153846153846156,
 0.5384615384615384,
 0.46153846153846156,
 0.46153846153846156,
 0.5384615384615384,
 0.46153846153846156,
 0.46153846153846156,
 0.5384615384615384,
 0.46153846153846156,
 0.46153846153846156,
 0.46153846153846156,
 0.3076923076923077,
 0.38461538461538464,
 0.23076923076923078,
 0.3076923076923077,
 0.3076923076923077,
 0.461

In [131]:
# Make pos neg pair for each row in anchor_pos_neg_triple (which is the pos neg based on cosine distances between attack and weakness nodes)

In [132]:
from collections import defaultdict
def get_cand(row, neg_min, pos_max):
    pos_candidates = [j for j, val in enumerate(row) if 0 <= val <= pos_max]
    neg_candidates = [j for j, val in enumerate(row) if neg_min <= val <= 1.0]
    return pos_candidates, neg_candidates
def find_pairs(dist_mat):
    pos_dict = defaultdict(list)
    neg_dict = defaultdict(list)

    for i, row in enumerate(dist_mat):
        neg_min=0.65
        pos_max=0.25
        pos_candidates, neg_candidates = get_cand(row, neg_min, pos_max)
        if len(pos_candidates)>10:
            pos_dict[i]=pos_candidates
        else:
            while(len(pos_candidates)<10):
                print("No pos for ", i, " posmax: ",pos_max, " ",pos_candidates)
                pos_max+=0.05
                pos_candidates, _ = get_cand(row, neg_min, pos_max)
            pos_dict[i]=pos_candidates
        if len(neg_candidates)>10:
            neg_dict[i]=neg_candidates
        else:
            while(len(neg_candidates)<10):
                #print("No neg for ", i, " negmin: ",neg_min, " ",neg_candidates)
                neg_min-=0.05
                _, neg_candidates = get_cand(row, neg_min, pos_max)
                #print(" new cand: ",neg_candidates)
            neg_dict[i]=neg_candidates
            

    return pos_dict, neg_dict


In [133]:
pos_dict,neg_dict= find_pairs(hop_dist_mat)

No pos for  0  posmax:  0.25   [1, 12]
No pos for  1  posmax:  0.25   [0, 23, 100, 138, 154, 185, 186, 187]
No pos for  3  posmax:  0.25   [2, 4, 32, 50, 65, 67, 70, 120, 178]
No pos for  8  posmax:  0.25   [9, 46, 55, 83, 86, 109, 123, 135]
No pos for  9  posmax:  0.25   [4, 8, 49, 50, 60, 77, 120, 138]
No pos for  13  posmax:  0.25   [14, 35, 48, 103, 132, 150]
No pos for  14  posmax:  0.25   [13, 34, 65, 67, 131]
No pos for  15  posmax:  0.25   [16, 45, 57, 105, 126, 160]
No pos for  16  posmax:  0.25   [15, 17, 20, 24, 104, 141]
No pos for  22  posmax:  0.25   [5, 10, 31, 69]
No pos for  23  posmax:  0.25   [1, 12, 39, 57, 72, 81, 106, 117]
No pos for  27  posmax:  0.25   [17, 18, 20, 50, 82, 92, 101, 120]
No pos for  32  posmax:  0.25   [3, 7, 33, 47, 148]
No pos for  35  posmax:  0.25   [13, 34, 49, 65, 67]
No pos for  36  posmax:  0.25   [37, 48, 93]
No pos for  37  posmax:  0.25   [34, 36, 100, 127, 131, 161, 168, 182]
No pos for  38  posmax:  0.25   [39, 57, 72, 106, 117]
No p

In [134]:
pos_dict[912]

[354,
 423,
 424,
 444,
 470,
 481,
 604,
 610,
 615,
 616,
 618,
 760,
 911,
 913,
 914,
 920,
 992,
 1026]

In [135]:
anchor_pos_neg_hop = []
for a,p,n in anchor_pos_neg_triple:
    hp = random.choice(pos_dict[a])
    hn = random.choice(neg_dict[a])
    anchor_pos_neg_hop.append((a,p,n,hp,hn))

In [136]:
anchor_pos_neg_hop[0]

(64, 689, 413, 18, 182)

In [137]:
with open(dir_name+'data/anchor_pos_neg_hop_{}.pkl'.format(approach), 'wb') as f:
    pickle.dump(anchor_pos_neg_hop,f)

In [ ]:
# Make pos neg pair for each row in training data (which is the hop distances between each attack and weakness nodes)

In [49]:
import random
pos_list = []
neg_list = []
for p1,_,_ in training_data:
    pos_neg_i = random.randrange(0, len(pos_neg_list[p1]))
    pos = pos_neg_list[p1][pos_neg_i][1]
    neg = pos_neg_list[p1][pos_neg_i][2]
    pos_list.append(pos)
    neg_list.append(neg)

In [52]:
pos_list = np.array(pos_list)
pos_list = pos_list.reshape((len(pos_list), 1))
pos_list.shape

(910562, 1)

In [53]:
neg_list = np.array(neg_list)
neg_list = neg_list.reshape((len(neg_list), 1))
neg_list.shape

(910562, 1)

In [54]:
training_data2 = np.concatenate((training_data, pos_list, neg_list), axis=1)


In [55]:
training_data2[:10]

array([[0.00000000e+00, 1.00000000e+00, 1.42857143e-01, 1.09200000e+03,
        5.94000000e+02],
       [1.00000000e+00, 0.00000000e+00, 1.42857143e-01, 3.04000000e+02,
        5.94000000e+02],
       [0.00000000e+00, 2.00000000e+00, 5.71428571e-01, 1.07900000e+03,
        1.05500000e+03],
       [2.00000000e+00, 0.00000000e+00, 5.71428571e-01, 5.30000000e+02,
        6.40000000e+02],
       [0.00000000e+00, 3.00000000e+00, 7.14285714e-01, 1.04200000e+03,
        6.45000000e+02],
       [3.00000000e+00, 0.00000000e+00, 7.14285714e-01, 6.09000000e+02,
        6.27000000e+02],
       [0.00000000e+00, 4.00000000e+00, 5.71428571e-01, 6.26000000e+02,
        1.02400000e+03],
       [4.00000000e+00, 0.00000000e+00, 5.71428571e-01, 6.04000000e+02,
        7.42000000e+02],
       [0.00000000e+00, 5.00000000e+00, 7.14285714e-01, 5.48000000e+02,
        3.95000000e+02],
       [5.00000000e+00, 0.00000000e+00, 7.14285714e-01, 7.47000000e+02,
        6.59000000e+02]])

In [51]:
neg_list[:10]

[594, 594, 1055, 640, 645, 627, 1024, 742, 395, 659]

In [56]:
np.save('../../ics_cwe/training_data_pair_pos_neg.npy',training_data2)